In [320]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re 

In [362]:
#Combines structured and unstrucutred data using BoW CountVectorizer 

def process_data(x, y):
    # Process Words
    x_words = x['first_name'].map(str) + ' ' + x['middle_name'].map(str)  + ' ' + x['lastname'].map(str) + ' ' + x['topics'].map(str)
    y_words = y['first_name'].map(str) + ' ' + y['middle_name'].map(str)  + ' ' + y['lastname'].map(str) + ' ' + y['topics'].map(str)
    
    #Total corpus is combining x and y
    total_BOW = pd.concat([x_words, y_words], sort=False)
    
    vectorizer = TfidfVectorizer()
    total_vec = vectorizer.fit(total_BOW)
    X = total_vec.transform(x_words).todense()
    X_words = pd.DataFrame(X)

    Y = total_vec.transform(y_words).todense()
    Y_words = pd.DataFrame(Y)
    
    #Process numbers
    x_num = pd.DataFrame()
    y_num = pd.DataFrame()
    

    x_num['cities'] =  [re.findall("(\d+)",i) for i in x['cities'].map(str)]
    x_num['countries'] =  [re.findall("(\d+)",i) for i in x['countries'].map(str)]
    x_num = x_num.fillna(0, inplace=True)

    y_num['cities'] =  [re.findall("(\d+)",x) for x in y['cities'].map(str)]
    y_num['countries'] =  [re.findall("(\d+)",x) for x in y['countries'].map(str)]
    y_num = y_num.fillna(0, inplace=True)
    
    return X_words.append(x_num), Y_words.append(y_num)
   # final_Y = pd.DataFrame(Y_words.append(y_num))
   # return final_X, final_Y

In [583]:
#returns euclidean distance with weighted feature if orcid are the same

def calc_euclid(x, y, final_X, final_Y, truth_table):
    distance = abs(np.linalg.norm(final_X.iloc[x] - final_Y.iloc[y]))
    if x in truth_table.id_x.tolist() and y == truth_df.id_y[truth_df[truth_df['id_x']==x].index.values.astype(int)[0]]:
        distance = distance - 10000
    return distance

In [578]:
# THis function creates truth table and includes euclid distance calculated

def create_truth_table(a, b, final_X, final_Y):
    dfa = a.drop_duplicates(subset=['orcid'])
    dfb = b.drop_duplicates(subset=['orcid'])
    dfa = dfa[np.isfinite(dfa['orcid'])]
    dfb = dfb[np.isfinite(dfb['orcid'])]
    new_df = pd.merge(dfa, dfb, how='inner', on='orcid')
    #new_df.drop(new_df.index[0], inplace=True)
    truth_df = new_df[['id_x','id_y']]
    
    #add this code in later to check euclid distance of truth table
    #truth_df['euclid'] = [calc_euclid(truth_df.id_x.iloc[i], truth_df.id_y.iloc[i], final_X, final_Y) for i in range(len(truth_df))]
    return truth_df

In [581]:
truth_df

,id_x,id_y
0,6,11
1,13,14
2,18,17
3,19,19
4,33,30
5,40,37
6,42,38
7,46,40
8,47,44
9,48,45


In [621]:
# Predict will run all of the functions in order and write a file to output
#predict(file_a, file_b):  
authors_df = pd.read_csv('authors_1019.csv', encoding='iso-8859-1')
investigators_df = pd.read_csv('investigators_837.csv', encoding='iso-8859-1')

final_X, final_Y = process_data(authors_df,investigators_df)

truth_df = create_truth_table(authors_df, investigators_df, final_X, final_Y)
results_df = pd.DataFrame()
results_df['author'] = authors_df.id-1
results_df.index = np.arange(0, len(results_df))


#export_csv = results_df.to_csv (r'ucb_output.csv', index = None, header=True)


In [622]:
results_df.head()

,author
0,0
1,1
2,2
3,3
4,4


In [624]:
final_Y.head()

,0,1,2,3,4,5,6,7,8,9,...,5827,5828,5829,5830,5831,5832,5833,5834,5835,5836
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [585]:
author_df.iloc[6]

NameError: name 'author_df' is not defined

In [415]:
print(calc_euclid(0, 1, final_X, final_Y, truth_df))

1.39968985732117


In [535]:
range(1, len(final_X)+1)

range(1, 1020)

In [604]:
result_list = []
euclid_list = []

for i in range(len(final_X)):
    min_index = 0
    min_euclid = 1000
    for j in range(len(final_Y)):
        x = calc_euclid(i, j, final_X, final_Y, truth_df)
        if x < min_euclid:
            min_euclid = x
            min_index = j
            print('i = ' + str(i) + ' j = ' + str(j) + ' min_euclid = ' + str(min_euclid) + ' min_index = ' + str(min_index))
    if i > 0:
        result_list.append(min_index)
        
result_list.append('1.2')
  #  euclid_list.append(min_euclid)

i = 0 j = 0 min_euclid = 1.4111254795004178 min_index = 0
i = 0 j = 1 min_euclid = 1.39968985732117 min_index = 1
i = 0 j = 6 min_euclid = 1.3915853446360023 min_index = 6
i = 0 j = 8 min_euclid = 1.3760935538276584 min_index = 8
i = 0 j = 23 min_euclid = 1.3531859941407074 min_index = 23
i = 0 j = 48 min_euclid = 1.325662075327123 min_index = 48
i = 0 j = 460 min_euclid = 1.3090657335331395 min_index = 460
i = 1 j = 0 min_euclid = 1.4107196028389353 min_index = 0
i = 1 j = 1 min_euclid = 1.3824274703927393 min_index = 1
i = 1 j = 4 min_euclid = 1.2581119652737025 min_index = 4
i = 1 j = 260 min_euclid = 1.250032221001067 min_index = 260
i = 1 j = 297 min_euclid = 1.2461586805943696 min_index = 297
i = 1 j = 531 min_euclid = 1.2436109198504726 min_index = 531
i = 1 j = 704 min_euclid = 1.2366526351604847 min_index = 704
i = 2 j = 0 min_euclid = 1.4084730861797525 min_index = 0
i = 2 j = 1 min_euclid = 1.4079265629702185 min_index = 1
i = 2 j = 2 min_euclid = 1.4076305929634307 min_inde

i = 22 j = 793 min_euclid = 1.2893677270927524 min_index = 793
i = 23 j = 0 min_euclid = 1.4100423394311354 min_index = 0
i = 23 j = 1 min_euclid = 1.3945674484751402 min_index = 1
i = 23 j = 23 min_euclid = 1.3669595139829567 min_index = 23
i = 23 j = 41 min_euclid = 1.3455196477028628 min_index = 41
i = 23 j = 138 min_euclid = 1.3189357430799564 min_index = 138
i = 23 j = 194 min_euclid = 1.305495249068216 min_index = 194
i = 23 j = 245 min_euclid = 1.2698685837753627 min_index = 245
i = 24 j = 0 min_euclid = 1.41120587128534 min_index = 0
i = 24 j = 1 min_euclid = 1.4000694571986452 min_index = 1
i = 24 j = 16 min_euclid = 1.3890809094331105 min_index = 16
i = 24 j = 23 min_euclid = 1.380289238684537 min_index = 23
i = 24 j = 48 min_euclid = 1.3721021276715408 min_index = 48
i = 24 j = 100 min_euclid = 1.3693541867736982 min_index = 100
i = 24 j = 138 min_euclid = 1.3691059316431429 min_index = 138
i = 24 j = 245 min_euclid = 1.3573151166848647 min_index = 245
i = 25 j = 0 min_eucli

i = 44 j = 323 min_euclid = 1.200346771998037 min_index = 323
i = 44 j = 347 min_euclid = 1.1942740591433492 min_index = 347
i = 45 j = 0 min_euclid = 1.3992407711708517 min_index = 0
i = 45 j = 1 min_euclid = 1.3986870229941737 min_index = 1
i = 45 j = 2 min_euclid = 1.3899024011399699 min_index = 2
i = 45 j = 4 min_euclid = 1.356783852620157 min_index = 4
i = 45 j = 16 min_euclid = 1.2797000645211913 min_index = 16
i = 45 j = 219 min_euclid = 1.2673356915443208 min_index = 219
i = 46 j = 0 min_euclid = 1.3893528553706291 min_index = 0
i = 46 j = 1 min_euclid = 1.3756039591217162 min_index = 1
i = 46 j = 40 min_euclid = -9998.610658768586 min_index = 40
i = 47 j = 0 min_euclid = 1.2979990393089156 min_index = 0
i = 47 j = 44 min_euclid = -9998.669253847285 min_index = 44
i = 48 j = 0 min_euclid = 1.4098419358340124 min_index = 0
i = 48 j = 1 min_euclid = 1.4015442970645857 min_index = 1
i = 48 j = 6 min_euclid = 1.3954752013448959 min_index = 6
i = 48 j = 8 min_euclid = 1.379629151425

i = 66 j = 0 min_euclid = 1.3860328852956438 min_index = 0
i = 66 j = 4 min_euclid = 1.3824495868572446 min_index = 4
i = 66 j = 5 min_euclid = 1.3729145224359416 min_index = 5
i = 66 j = 16 min_euclid = 1.3647744414657406 min_index = 16
i = 66 j = 18 min_euclid = 1.364268877496166 min_index = 18
i = 66 j = 27 min_euclid = 1.345376495597823 min_index = 27
i = 66 j = 48 min_euclid = 1.3437504501040058 min_index = 48
i = 66 j = 60 min_euclid = 1.3361271722790402 min_index = 60
i = 66 j = 271 min_euclid = 1.3197222751270166 min_index = 271
i = 67 j = 0 min_euclid = 1.3662134187863673 min_index = 0
i = 67 j = 6 min_euclid = 1.3491355893713037 min_index = 6
i = 67 j = 8 min_euclid = 1.3221171230980113 min_index = 8
i = 67 j = 62 min_euclid = -9998.597287214416 min_index = 62
i = 68 j = 0 min_euclid = 1.4111476347783851 min_index = 0
i = 68 j = 1 min_euclid = 1.405336146407968 min_index = 1
i = 68 j = 23 min_euclid = 1.381240978455964 min_index = 23
i = 68 j = 48 min_euclid = 1.3790250592266

i = 84 j = 382 min_euclid = 1.2628005770930744 min_index = 382
i = 85 j = 0 min_euclid = 1.4047700595350765 min_index = 0
i = 85 j = 1 min_euclid = 1.3806735158081174 min_index = 1
i = 85 j = 5 min_euclid = 1.3760801455409046 min_index = 5
i = 85 j = 16 min_euclid = 1.3725308558346405 min_index = 16
i = 85 j = 23 min_euclid = 1.3409680312104806 min_index = 23
i = 85 j = 48 min_euclid = 1.3263996850522275 min_index = 48
i = 85 j = 100 min_euclid = 1.3197231369225937 min_index = 100
i = 85 j = 245 min_euclid = 1.2866180204133957 min_index = 245
i = 86 j = 0 min_euclid = 1.4095452258166092 min_index = 0
i = 86 j = 1 min_euclid = 1.4006816875863555 min_index = 1
i = 86 j = 8 min_euclid = 1.4006165666291785 min_index = 8
i = 86 j = 14 min_euclid = 1.3963257861376732 min_index = 14
i = 86 j = 23 min_euclid = 1.3829968474567442 min_index = 23
i = 86 j = 48 min_euclid = 1.3690740562898116 min_index = 48
i = 86 j = 245 min_euclid = 1.3630901403567 min_index = 245
i = 86 j = 467 min_euclid = 1.3

...

i = 940 j = 792 min_euclid = -9998.828622630745 min_index = 792
i = 941 j = 0 min_euclid = 1.4002211828209021 min_index = 0
i = 941 j = 1 min_euclid = 1.3941514505677919 min_index = 1
i = 941 j = 4 min_euclid = 1.3929776965861351 min_index = 4
i = 941 j = 6 min_euclid = 1.3397969387606243 min_index = 6
i = 941 j = 40 min_euclid = 1.3174298841366352 min_index = 40
i = 941 j = 65 min_euclid = 1.3115911353316394 min_index = 65
i = 941 j = 128 min_euclid = 1.2763634706267895 min_index = 128
i = 941 j = 305 min_euclid = 1.263905234615582 min_index = 305
i = 941 j = 489 min_euclid = 1.2515518172085345 min_index = 489
i = 941 j = 793 min_euclid = -9998.775745919726 min_index = 793
i = 942 j = 0 min_euclid = 1.3744128551932546 min_index = 0
i = 942 j = 5 min_euclid = 1.373632082195647 min_index = 5
i = 942 j = 6 min_euclid = 1.3453689118545757 min_index = 6
i = 942 j = 18 min_euclid = 1.2979930162260624 min_index = 18
i = 942 j = 128 min_euclid = 1.2881081759622393 min_index = 128
i = 942 j = 

i = 955 j = 793 min_euclid = 1.2894384938608459 min_index = 793
i = 956 j = 0 min_euclid = 1.4099986846393668 min_index = 0
i = 956 j = 1 min_euclid = 1.4019998636428457 min_index = 1
i = 956 j = 23 min_euclid = 1.3950109408193587 min_index = 23
i = 956 j = 44 min_euclid = 1.3015914401247781 min_index = 44
i = 956 j = 246 min_euclid = 1.2852844304190418 min_index = 246
i = 956 j = 578 min_euclid = 1.2747645224751822 min_index = 578
i = 956 j = 813 min_euclid = -9998.585786437627 min_index = 813
i = 957 j = 0 min_euclid = 1.4113146920163857 min_index = 0
i = 957 j = 1 min_euclid = 1.4058208082308044 min_index = 1
i = 957 j = 6 min_euclid = 1.4025598360220612 min_index = 6
i = 957 j = 8 min_euclid = 1.399408273085366 min_index = 8
i = 957 j = 14 min_euclid = 1.396407079699702 min_index = 14
i = 957 j = 18 min_euclid = 1.340708503862604 min_index = 18
i = 957 j = 649 min_euclid = 1.3272854368815072 min_index = 649
i = 958 j = 0 min_euclid = 1.4088479313000135 min_index = 0
i = 958 j = 1 m

i = 970 j = 286 min_euclid = 1.242943311962719 min_index = 286
i = 970 j = 467 min_euclid = 1.1787881842295416 min_index = 467
i = 971 j = 0 min_euclid = 1.3899152990847456 min_index = 0
i = 971 j = 11 min_euclid = 1.3873503380711452 min_index = 11
i = 971 j = 48 min_euclid = 1.378767891476626 min_index = 48
i = 971 j = 73 min_euclid = 1.3743472013878228 min_index = 73
i = 971 j = 350 min_euclid = 1.371825905356844 min_index = 350
i = 971 j = 405 min_euclid = 1.3664680927046464 min_index = 405
i = 971 j = 824 min_euclid = 1.3530131865966775 min_index = 824
i = 972 j = 0 min_euclid = 1.4095185942673891 min_index = 0
i = 972 j = 1 min_euclid = 1.4006042459799173 min_index = 1
i = 972 j = 8 min_euclid = 1.400538750513653 min_index = 8
i = 972 j = 14 min_euclid = 1.3962232547583302 min_index = 14
i = 972 j = 18 min_euclid = 1.3324777930885199 min_index = 18
i = 972 j = 23 min_euclid = 1.3229536200339371 min_index = 23
i = 972 j = 42 min_euclid = 1.3084935079556275 min_index = 42
i = 972 j 

i = 987 j = 25 min_euclid = 1.3506228202392003 min_index = 25
i = 987 j = 110 min_euclid = 1.3302780486259622 min_index = 110
i = 987 j = 159 min_euclid = 1.3032117705876838 min_index = 159
i = 988 j = 0 min_euclid = 1.4101824467083983 min_index = 0
i = 988 j = 1 min_euclid = 1.4025338161986765 min_index = 1
i = 988 j = 6 min_euclid = 1.3918414463654771 min_index = 6
i = 988 j = 8 min_euclid = 1.3846477519683376 min_index = 8
i = 988 j = 14 min_euclid = 1.3799593799066325 min_index = 14
i = 988 j = 16 min_euclid = 1.3453124646693135 min_index = 16
i = 988 j = 48 min_euclid = 1.3422229604058011 min_index = 48
i = 988 j = 124 min_euclid = 1.3309807516454308 min_index = 124
i = 988 j = 218 min_euclid = 1.3286869218541248 min_index = 218
i = 988 j = 250 min_euclid = 1.328660815600655 min_index = 250
i = 988 j = 710 min_euclid = 1.3201692706511428 min_index = 710
i = 989 j = 0 min_euclid = 1.4027892113995546 min_index = 0
i = 989 j = 1 min_euclid = 1.3851191871952584 min_index = 1
i = 989 j

i = 1004 j = 333 min_euclid = 1.3114691542998984 min_index = 333
i = 1004 j = 496 min_euclid = 1.305976957803657 min_index = 496
i = 1004 j = 668 min_euclid = 1.2771704421096028 min_index = 668
i = 1004 j = 785 min_euclid = 1.2525679741714857 min_index = 785
i = 1005 j = 0 min_euclid = 1.410056688907185 min_index = 0
i = 1005 j = 1 min_euclid = 1.3946354075396354 min_index = 1
i = 1005 j = 3 min_euclid = 1.254836413560659 min_index = 3
i = 1005 j = 834 min_euclid = 1.2016385594954935 min_index = 834
i = 1006 j = 0 min_euclid = 1.41043722000895 min_index = 0
i = 1006 j = 1 min_euclid = 1.403273883123807 min_index = 1
i = 1006 j = 23 min_euclid = 1.3970184869229116 min_index = 23
i = 1006 j = 34 min_euclid = 1.3434141669251294 min_index = 34
i = 1006 j = 78 min_euclid = 1.3320179978530493 min_index = 78
i = 1006 j = 833 min_euclid = 1.3080017763278258 min_index = 833
i = 1006 j = 834 min_euclid = 1.3003048186824504 min_index = 834
i = 1007 j = 0 min_euclid = 1.414213562373095 min_index =

In [606]:
len(result_list)

1019

In [617]:
results_df['investigator'] = result_list
results_df['author'] += 1


In [619]:
results_df.head(20)

,author,investigator
0,1,704
1,2,570
2,3,234
3,4,360
4,5,10
5,6,11
6,7,245
7,8,386
8,9,262
9,10,159


In [620]:
#accuracy assesment on truth data
score = 0
incorrect_x = []
incorrect_y = []
results_inv = []

for x in range(len(truth_df)):
    index_y = 0
    index_y = truth_df.iloc[x].id_x - 1
    if truth_df.iloc[x].id_y == results_df.iloc[index_y].investigator:
        score +=1
    elif truth_df.iloc[x].id_y != results_df.iloc[index_y].investigator:
        incorrect_x.append(truth_df.iloc[x].id_x)
        incorrect_y.append(truth_df.iloc[x].id_y)
        results_inv.append(results_df.iloc[index_y].investigator)

print('Accuracy on ground truth is ' + str(score / len(truth_df)*100) + '%')

Accuracy on ground truth is 100.0%


In [506]:
incorrect_df = pd.DataFrame([incorrect_x, incorrect_y, results_inv]).transpose()
incorrect_df.columns = ['true_author','true_investigator', 'incorrect_investigator']
incorrect_df.head()

,true_author,true_investigator,incorrect_investigator
0,18,17,258
1,19,19,18
2,33,30,713
3,42,38,555
4,46,40,220


In [452]:
print(calc_euclid(6, 11, final_X, final_Y, truth_df))
print(calc_euclid(19, 19, final_X, final_Y, truth_df))

weight function is working. x = 6 y = 11
-9998.615988840156
weight function is working. x = 19 y = 19
-9998.590565579158


In [421]:
truth_df.shape

(206, 2)

In [428]:
truth_df[truth_df['id_x']==19].index.values.astype(int)[0]

4

In [430]:
truth_df.id_y[truth_df[truth_df['id_x']==19].index.values.astype(int)[0]]

19

In [528]:
result_list = []
euclid_list = []

for i in range(1,20):
    print('i =' + str(i))
    min_index = 0
    min_euclid = 1000
    for j in range(1, len(final_Y)):
        x = calc_euclid(i, j, final_X, final_Y, truth_df)
        
        if x < min_euclid:
            #print('min_euclid' + str(min_euclid) + ' and index is ' + str(min_index))
            min_euclid = x
            min_index = j
            print('i = ' + str(i) + 'x = ' + str(x) + ' and min_euclid = ' + str(min_euclid) + ' j = ' + str(j) + ' min_index = ' + str(min_index))
    result_list.append(min_index)
    euclid_list.append(min_euclid)

i =1
i = 1x = 1.3824274703927393 and min_euclid = 1.3824274703927393 j = 1 min_index = 1
i = 1x = 1.2581119652737025 and min_euclid = 1.2581119652737025 j = 4 min_index = 4
i = 1x = 1.250032221001067 and min_euclid = 1.250032221001067 j = 260 min_index = 260
i = 1x = 1.2461586805943696 and min_euclid = 1.2461586805943696 j = 297 min_index = 297
i = 1x = 1.2436109198504726 and min_euclid = 1.2436109198504726 j = 531 min_index = 531
i = 1x = 1.2366526351604847 and min_euclid = 1.2366526351604847 j = 704 min_index = 704
i =2
i = 2x = 1.4079265629702185 and min_euclid = 1.4079265629702185 j = 1 min_index = 1
i = 2x = 1.4076305929634307 and min_euclid = 1.4076305929634307 j = 2 min_index = 2
i = 2x = 1.3992369936323392 and min_euclid = 1.3992369936323392 j = 3 min_index = 3
i = 2x = 1.3662153388108385 and min_euclid = 1.3662153388108385 j = 570 min_index = 570
i =3
i = 3x = 1.4014691343444694 and min_euclid = 1.4014691343444694 j = 1 min_index = 1
i = 3x = 1.3941568128109316 and min_euclid 

In [529]:
result_list

[704,
 570,
 234,
 360,
 10,
 11,
 245,
 386,
 262,
 159,
 12,
 13,
 14,
 347,
 578,
 15,
 257,
 17,
 19]

In [402]:
def calc_euclid_test(x, y, final_X, final_Y, truth_table):
    distance = abs(np.linalg.norm(final_X.iloc[x] - final_Y.iloc[y]))
#     print(x)
#     print(truth_table.id_x)
    if x in truth_table.id_x.tolist() and y in truth_table.id_y.tolist():
        print('weight function is working')
        distance = distance - 10000
    return distance

In [466]:
truth_df.head(20)

,id_x,id_y
1,6,11
2,13,14
3,18,17
4,19,19
5,33,30
6,40,37
7,42,38
8,46,40
9,47,44
10,48,45


In [54]:
predict('authors_1019.csv','investigators_837.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
